# Simple Filtering

## Summary
In this notebook, we'll be covering:
- [Filtering using mathematical operators](#Filtering-Using-Mathematical-Operators)
- [Filtering using isin()](#Filtering-Using-isin())
- [Chaining filters](#Chaining-Filters)

In this section we will discuss filtering data. The All of Us Researcher Workbench will do *some* filtering for you, but undoubtedly, you will need to perform additional filtering to conduct your analysis. Filtering can become complex, so this section is split into two notebooks. This one covers simple filters, while the next one covers creating new data labels that we can then use to further subset our data.

### Filtering Using Mathematical Operators
To get started with filtering, we first need a dataset to use for demonstration. The code below will make a dataframe for us. In fact, it will be the same dataframe as the previous notebook, but without random blank cells.

It is not important to understand all of the code in the cell below. The important thing to know is that we are generating fake data to use for demonstration. Our fake dataset will have the following columns:

* ID
* Measurement Device
* Heart Rate Max
* Heart Rate Min
* Heart Rate Avg
* Duration of exercise (min)
* Exercise Type

In [1]:
import pandas as pd
import random

workout_dict = {'ID': [], 'Measurement Device': [], 'Heart Rate Max': [], 'Heart Rate Min': [], 'Heart Rate Avg': [],
              'Duration of exercise (min)': [], 'Exercise Type': []}
used_ids = []

for x in range(0, 500):
    id = random.randint(100000000, 999999999)
    while id in used_ids:
        id = random.randint(100000000, 999999999)
    used_ids.append(id)
    device = random.choice(['Skykandal', 'B-Wolf'])
    mu = random.randint(65, 85)
    min_rate = int(random.gauss(mu, 10))
    max_rate = int(random.gauss(mu + 55, 25))
    while max_rate <= min_rate:
        max_rate = int(random.gauss(mu + 55, 25))
    avg = random.gauss((max_rate + min_rate) / 2, (max_rate - min_rate) / 5)
    duration = random.randint(10, 90)
    exercise = random.choice(['Running', 'Running', 'Running', 'Bicycling', 'Swimming', 'Swimming',
                              'Weight training'])
    row = [device, min_rate, max_rate, avg, duration, exercise]
    workout_dict['ID'].append(id)
    workout_dict['Measurement Device'].append(row[0])
    workout_dict['Heart Rate Min'].append(row[1])
    workout_dict['Heart Rate Max'].append(row[2])
    workout_dict['Heart Rate Avg'].append(row[3])
    workout_dict['Duration of exercise (min)'].append(row[4])
    workout_dict['Exercise Type'].append(row[5])

df = pd.DataFrame(workout_dict)
df.head(10)

,ID,Measurement Device,Heart Rate Max,Heart Rate Min,Heart Rate Avg,Duration of exercise (min),Exercise Type
0,967845598,Skykandal,122,83,86.396160,13,Running
1,738520136,B-Wolf,121,74,87.761404,46,Weight training
2,416079826,Skykandal,103,76,98.886300,42,Running
3,166760069,B-Wolf,119,97,110.594568,67,Bicycling
4,697975177,B-Wolf,106,91,96.235489,60,Running
5,110077262,Skykandal,154,60,95.631071,89,Running
6,764988594,B-Wolf,156,59,123.673281,68,Weight training
7,158317145,Skykandal,126,73,103.042381,83,Weight training
8,230824273,Skykandal,118,77,97.940015,62,Running
9,503405497,Skykandal,118,75,96.283207,17,Running


Let's start with a simple filter: we will select everyone whose **minimum heart rate** is greater than 69.

In [2]:
df[df['Heart Rate Min'] > 69]

,ID,Measurement Device,Heart Rate Max,Heart Rate Min,Heart Rate Avg,Duration of exercise (min),Exercise Type
0,967845598,Skykandal,122,83,86.396160,13,Running
1,738520136,B-Wolf,121,74,87.761404,46,Weight training
2,416079826,Skykandal,103,76,98.886300,42,Running
3,166760069,B-Wolf,119,97,110.594568,67,Bicycling
4,697975177,B-Wolf,106,91,96.235489,60,Running
...,...,...,...,...,...,...,...
494,638182753,B-Wolf,190,88,143.298550,79,Bicycling
495,620830646,Skykandal,144,80,103.424546,68,Bicycling
496,906832067,B-Wolf,149,100,131.675240,69,Running
497,950761576,Skykandal,150,92,131.812747,24,Weight training


How does this work? Well, we start with `df[]` which is a common way to get something out of dataframe `df` using keywords. Except, instead of passing something like a column name, we passed an expression into the brackets. The expression was `df['Heart Rate Min'] > 69'`. In other words, from df give me every row where Heart Rate Min (in df) is greater than 69.

Now, let's get one of the most annoying features of pandas out of the way. We're going to store our filtered data in df_filtered and then, for no good reason, change a column.

In [3]:
df_filtered = df[df['Heart Rate Min'] > 69]
df_filtered['Heart Rate Avg'] = 200
df_filtered.head()

/var/folders/j7/87lgk7nd3t7d39tm0qm2s4n40000gn/T/ipykernel_52843/3099549399.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Heart Rate Avg'] = 200


,ID,Measurement Device,Heart Rate Max,Heart Rate Min,Heart Rate Avg,Duration of exercise (min),Exercise Type
0,967845598,Skykandal,122,83,200,13,Running
1,738520136,B-Wolf,121,74,200,46,Weight training
2,416079826,Skykandal,103,76,200,42,Running
3,166760069,B-Wolf,119,97,200,67,Bicycling
4,697975177,B-Wolf,106,91,200,60,Running


Above you should have gotten a `SettingWithCopyWarning`. You might have assumed that when you assigned `df[df['Heart Rate Min'] > 69]` to `df_filtered` you created an entirely new dataframe, named `df_filtered`. You didn't. You created a "view" of the old dataframe (`df`) that only saw the parts of `df` that you wanted, but wasn't actually a new dataframe. This means that attempting to set a value in `df_filtered` may also alter values in `df`. And so we get a warning.

When you get a view versus when you get a new pandas object is fairly complicated. However, there is a simple fix: `copy` can turn a view into a copy. Copy is a dataframe method, and takes no arguments.

Below, the code has added `copy()` and what previously gave us a warning now won't. To be clear, the main difference is that now `df_filtered` is a *new dataframe* instead of a view of the original dataframe. 

In [4]:
df_filtered = df[df['Heart Rate Min'] > 69].copy()
df_filtered['Heart Rate Avg'] = 200
df_filtered.head()

,ID,Measurement Device,Heart Rate Max,Heart Rate Min,Heart Rate Avg,Duration of exercise (min),Exercise Type
0,967845598,Skykandal,122,83,200,13,Running
1,738520136,B-Wolf,121,74,200,46,Weight training
2,416079826,Skykandal,103,76,200,42,Running
3,166760069,B-Wolf,119,97,200,67,Bicycling
4,697975177,B-Wolf,106,91,200,60,Running


Filtering expressions can include a number of comparison operators, such as: >, <, <=, >=, ==, and !=. Remember that in Python = means "set the thing on the left side of the = equal to the thing on the right side of the =", whereas == asks if they are equal. != means "not equal".

These expressions are fairly straightforward when we are dealing with numbers. 

#### Below, write some code that filters `df` so that only individuals with a maximum heart rate of 110 or under remain.

In [5]:
# your code goes here


You can also use some of these operators, like == and != to filter on text columns. Below, we'll just select the individuals who went running.

In [6]:
df[df['Exercise Type'] == 'Running']

,ID,Measurement Device,Heart Rate Max,Heart Rate Min,Heart Rate Avg,Duration of exercise (min),Exercise Type
0,967845598,Skykandal,122,83,86.396160,13,Running
2,416079826,Skykandal,103,76,98.886300,42,Running
4,697975177,B-Wolf,106,91,96.235489,60,Running
5,110077262,Skykandal,154,60,95.631071,89,Running
8,230824273,Skykandal,118,77,97.940015,62,Running
...,...,...,...,...,...,...,...
490,506553868,Skykandal,136,87,115.893668,25,Running
492,871880249,B-Wolf,131,41,102.196239,40,Running
496,906832067,B-Wolf,149,100,131.675240,69,Running
498,838376370,B-Wolf,128,58,97.545689,40,Running


We could also *exclud*e people who went running using `~`. This means "not", and it's generally good practice (i.e., it will sometimes break if you don't do this) to wrap the expression you are negating in parentheses, as below.

In [7]:
df[~(df['Exercise Type'] == 'Running')]

,ID,Measurement Device,Heart Rate Max,Heart Rate Min,Heart Rate Avg,Duration of exercise (min),Exercise Type
1,738520136,B-Wolf,121,74,87.761404,46,Weight training
3,166760069,B-Wolf,119,97,110.594568,67,Bicycling
6,764988594,B-Wolf,156,59,123.673281,68,Weight training
7,158317145,Skykandal,126,73,103.042381,83,Weight training
11,115382296,Skykandal,115,76,96.841052,34,Swimming
...,...,...,...,...,...,...,...
491,569794385,Skykandal,138,81,108.865881,20,Bicycling
493,493299437,B-Wolf,142,80,98.976302,28,Swimming
494,638182753,B-Wolf,190,88,143.298550,79,Bicycling
495,620830646,Skykandal,144,80,103.424546,68,Bicycling


What happens if we use a > or < symbol?

In [8]:
df[df['Exercise Type'] < 'Running']

,ID,Measurement Device,Heart Rate Max,Heart Rate Min,Heart Rate Avg,Duration of exercise (min),Exercise Type
3,166760069,B-Wolf,119,97,110.594568,67,Bicycling
22,586661873,B-Wolf,111,92,96.683467,23,Bicycling
26,331387570,Skykandal,106,56,79.920947,82,Bicycling
27,334299313,B-Wolf,118,93,108.350289,20,Bicycling
52,608904153,Skykandal,154,57,133.234176,79,Bicycling
...,...,...,...,...,...,...,...
488,978175076,Skykandal,146,90,113.161519,69,Bicycling
489,567493772,Skykandal,144,74,94.875436,78,Bicycling
491,569794385,Skykandal,138,81,108.865881,20,Bicycling
494,638182753,B-Wolf,190,88,143.298550,79,Bicycling


`< 'Running'` gives us only bicyclists, because > and < on text check alphabetization. < 'Running' gave us every label that is, when alphabetized, earlier than "Running". This behavior is rarely useful. It's generally best practice to avoid '<' and '>' with string (i.e., text) columns. 

#### Below, write code that returns only people using the B-Wolf measuring device. Do this two different ways. To make it easier to see that each way worked there are separate cells for each way.

In [9]:
# your code goes here for the first way


In [10]:
# the second way goes here


What if we wanted to filter based on the relationship between two columns? We can pass more complex expressions to the filter as well. The filter below will return individuals whose maximum heart rate is within 20 beats of their minimum heart rate.

In [11]:
df[(df['Heart Rate Max'] - df['Heart Rate Min']) <= 20]

,ID,Measurement Device,Heart Rate Max,Heart Rate Min,Heart Rate Avg,Duration of exercise (min),Exercise Type
4,697975177,B-Wolf,106,91,96.235489,60,Running
10,358607893,Skykandal,106,87,96.044343,87,Running
19,835069760,Skykandal,89,71,83.973844,58,Running
22,586661873,B-Wolf,111,92,96.683467,23,Bicycling
29,705516118,B-Wolf,115,95,104.672788,70,Running
40,909475531,B-Wolf,92,76,87.460360,81,Swimming
51,248006541,B-Wolf,92,88,91.625448,67,Running
54,822622139,Skykandal,116,106,110.119222,10,Running
55,750231831,Skykandal,79,72,73.973507,80,Running
76,350606231,B-Wolf,70,65,66.939569,12,Running


#### Below, write code that returns everyone whose average heart rate was less than the number you would get by averaging the maximum and minimum heart rates. It's easy to miss parentheses and brackets when writing something this long, so I suggest that you write in your parentheses/brackets first, and then fill them in.

In [12]:
# your code goes here


### Filtering Using isin()
The filtering tools above work fine for text columns where we want only one value, but what if we want more than one value? The `isin` method is our answer.

In [13]:
df[df['Exercise Type'].isin(['Running', 'Swimming'])]

,ID,Measurement Device,Heart Rate Max,Heart Rate Min,Heart Rate Avg,Duration of exercise (min),Exercise Type
0,967845598,Skykandal,122,83,86.396160,13,Running
2,416079826,Skykandal,103,76,98.886300,42,Running
4,697975177,B-Wolf,106,91,96.235489,60,Running
5,110077262,Skykandal,154,60,95.631071,89,Running
8,230824273,Skykandal,118,77,97.940015,62,Running
...,...,...,...,...,...,...,...
492,871880249,B-Wolf,131,41,102.196239,40,Running
493,493299437,B-Wolf,142,80,98.976302,28,Swimming
496,906832067,B-Wolf,149,100,131.675240,69,Running
498,838376370,B-Wolf,128,58,97.545689,40,Running


The `isin` method is very similar to the basic Python `in`. It simply checks to see if something is in an iterable that you provide. In this case we simply passed the list `['Running', 'Swimming']` to `isin` and got back every row where the value in the Condition column was in the list.

If we had a lot of categories and we wanted to get most, but not all, of them we could in theory use `isin` and write a long list of categories to be included. However, it would be nicer to exclude the small list. Thankfully, ~ works for `isin` just like it did for mathematical expressions.

Below, we write an expression to return all the rows where the Exercise Type is **not** in the list `['Running', 'Swimming']`.

In [14]:
df[~df['Exercise Type'].isin(['Running', 'Swimming'])]

,ID,Measurement Device,Heart Rate Max,Heart Rate Min,Heart Rate Avg,Duration of exercise (min),Exercise Type
1,738520136,B-Wolf,121,74,87.761404,46,Weight training
3,166760069,B-Wolf,119,97,110.594568,67,Bicycling
6,764988594,B-Wolf,156,59,123.673281,68,Weight training
7,158317145,Skykandal,126,73,103.042381,83,Weight training
22,586661873,B-Wolf,111,92,96.683467,23,Bicycling
...,...,...,...,...,...,...,...
489,567493772,Skykandal,144,74,94.875436,78,Bicycling
491,569794385,Skykandal,138,81,108.865881,20,Bicycling
494,638182753,B-Wolf,190,88,143.298550,79,Bicycling
495,620830646,Skykandal,144,80,103.424546,68,Bicycling


While you might be tempted to try and write `df.~isin` Python interprets that as nonsense. In reality, `df.isin` is creating a dataframe full of `True` and `False`, and we're asking for that frame to be flipped, so every `True` is a `False` and vice-versa.

#### For a bit of quick practice, make a filter that returns everyone who wasn't bicycling or running.

In [15]:
# your code goes here


### Chaining Filters
Now, we can combine filters by chaining filters together in the following manner:

In [16]:
temp_df = df[df['Heart Rate Min'] > 80]
temp_df[temp_df['Measurement Device'] == 'Skykandal']

,ID,Measurement Device,Heart Rate Max,Heart Rate Min,Heart Rate Avg,Duration of exercise (min),Exercise Type
0,967845598,Skykandal,122,83,86.396160,13,Running
10,358607893,Skykandal,106,87,96.044343,87,Running
12,514637587,Skykandal,150,104,151.521491,27,Swimming
20,555816001,Skykandal,124,93,114.359236,60,Swimming
32,448642180,Skykandal,129,85,91.647146,41,Swimming
...,...,...,...,...,...,...,...
473,575424347,Skykandal,176,83,122.097748,87,Weight training
488,978175076,Skykandal,146,90,113.161519,69,Bicycling
490,506553868,Skykandal,136,87,115.893668,25,Running
491,569794385,Skykandal,138,81,108.865881,20,Bicycling


However, this is ugly and requires the creation of potentially a large number of temporary dataframes that exist only to hold intermediate steps. We could do the same thing a different way: write each expression in parentheses and then connect them together.

Below is the same result as above but in one line.

In [17]:
df[(df['Heart Rate Min'] > 80) & (df['Measurement Device'] == 'Skykandal')]

,ID,Measurement Device,Heart Rate Max,Heart Rate Min,Heart Rate Avg,Duration of exercise (min),Exercise Type
0,967845598,Skykandal,122,83,86.396160,13,Running
10,358607893,Skykandal,106,87,96.044343,87,Running
12,514637587,Skykandal,150,104,151.521491,27,Swimming
20,555816001,Skykandal,124,93,114.359236,60,Swimming
32,448642180,Skykandal,129,85,91.647146,41,Swimming
...,...,...,...,...,...,...,...
473,575424347,Skykandal,176,83,122.097748,87,Weight training
488,978175076,Skykandal,146,90,113.161519,69,Bicycling
490,506553868,Skykandal,136,87,115.893668,25,Running
491,569794385,Skykandal,138,81,108.865881,20,Bicycling


All we've done here is connect two expressions we already understand with &. Python evaluates each expression (`df['Heart Rate Min'] > 80` and `df['Measurement Device'] == 'Skykandal'`) separately. If they both evaluate to True then the whole statement evaluates to True.

#### Write some code below that filters for all people who were swimming and spent more than 60 minutes doing so.

In [18]:
# your code goes here


There are other ways to connect expressions. `&` handles "and", meaning both parts have to be true. `|` (which you can generally type as Shift+\\) means "or", meaning at least one of the parts has to be true.

This is how `&` and `|` interpret logical expressions:
* True & True = True
* False & True = False
* False | True = True
* True | True = True

Between `&`, `|`, and `~` you can build some fairly complex expressions, especially since you can connect blocks inside parentheses with other blocks.

The code block below filters for everyone with an average heart rate of 120-130 or 140-150. Visually, the line is broken at the | so you can more easily see the two halves.

In [19]:
df[((df['Heart Rate Avg']>= 120) & (df['Heart Rate Avg'] <= 130)) | 
   ((df['Heart Rate Avg'] >= 140) & (df['Heart Rate Avg'] <= 150))]

,ID,Measurement Device,Heart Rate Max,Heart Rate Min,Heart Rate Avg,Duration of exercise (min),Exercise Type
6,764988594,B-Wolf,156,59,123.673281,68,Weight training
15,303149860,Skykandal,152,77,125.059724,81,Swimming
18,831147960,B-Wolf,142,75,146.210892,75,Swimming
21,491926925,B-Wolf,156,65,147.633918,41,Running
37,348508933,B-Wolf,146,79,122.624314,68,Swimming
38,881939803,Skykandal,133,58,120.210398,16,Swimming
46,521997819,Skykandal,140,88,126.783743,45,Weight training
59,744740160,B-Wolf,161,76,125.860468,87,Running
69,232370085,B-Wolf,176,90,146.683107,14,Running
71,407527101,B-Wolf,144,85,128.190240,20,Swimming


There's more than one way to do this. The code block below will give the same results as the one above. Spend a minute looking at it until you can tell why. It may help to think about both filters in chunks in pieces (both filters are two sets of two linked expressions).

In [20]:
df[~((df['Heart Rate Avg'] < 120) | (df['Heart Rate Avg'] > 150)) &
   ~((df['Heart Rate Avg'] > 130) & (df['Heart Rate Avg'] < 140))]

,ID,Measurement Device,Heart Rate Max,Heart Rate Min,Heart Rate Avg,Duration of exercise (min),Exercise Type
6,764988594,B-Wolf,156,59,123.673281,68,Weight training
15,303149860,Skykandal,152,77,125.059724,81,Swimming
18,831147960,B-Wolf,142,75,146.210892,75,Swimming
21,491926925,B-Wolf,156,65,147.633918,41,Running
37,348508933,B-Wolf,146,79,122.624314,68,Swimming
38,881939803,Skykandal,133,58,120.210398,16,Swimming
46,521997819,Skykandal,140,88,126.783743,45,Weight training
59,744740160,B-Wolf,161,76,125.860468,87,Running
69,232370085,B-Wolf,176,90,146.683107,14,Running
71,407527101,B-Wolf,144,85,128.190240,20,Swimming


Remember how I said that making intermediate data frames is bad practice? When the expression gets complicated enough, you sometimes need to perform intermediate steps for debugging purposes. However, you can potentially build extremely powerful filters this way.

#### Below, write code that filters for people who went running or swimming while using the B-Wolf device OR who went bicycling using Skykandal, and have an average heart rate below the number you get by averaging their maximum and minimum heart rates together.

In [21]:
# your code goes here


At this point you may be thinking, "Wow, I wish I could write a function, pass all of these rows to that function, evaluate the row there, and then just return a True/False label I could filter against."

That's a great idea! We'll cover it in the next notebook.